## Imports

In [ ]:
import pandas as pd

In [ ]:
day = "12"

In [ ]:
df = pd.read_csv(f"input_j{day}.txt", header=None, sep=",")
# liste = list(df[1])

len(df)

In [ ]:
# with open(f"input_j{day}.txt") as f:
#     infos = f.readlines()
#     infos = list(map(lambda x: x.replace("\n", ""), infos))

## Part 1